# LSTM Ten-year Italy yield

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from tqdm import tqdm
from keras.models import Sequential,Input,Model
from keras.layers import LSTM,Conv2D,Flatten
from keras.layers import Dense,Dropout
import copy

Using TensorFlow backend.


In [ ]:
daily_data = pd.read_csv("Rendimenti_italia_giornaliero.csv",header=None,sep=";")
weekly_data = pd.read_csv("Rendimenti_italia.csv",header=None,sep=";")
germany_daily_data = pd.read_csv("Rendimenti_germania_giornaliero.csv",header=None,sep=";")

In [ ]:
weekly_data=weekly_data[::-1]
weekly_data.columns=['Data','Close','Open','High','Low','Variation']
weekly_data.head()

In [ ]:
fig = go.Figure(data=[go.Candlestick(x=weekly_data['Data'],
                open=weekly_data['Open'], high=weekly_data['High'],
                low=weekly_data['Low'], close=weekly_data['Close'])])
fig.update_layout(
    title='Ten-year Italy yield',
    yaxis_title='',
    width=768)
fig.show()

In [ ]:
daily_data=daily_data[::-1]
daily_data.columns=['Data','Close','Open','High','Low','Variation']
daily_data.head()

In [ ]:
germany_daily_data=germany_daily_data[::-1]
germany_daily_data.columns=['Data','Close','Open','High','Low','Variation']
i=germany_daily_data[germany_daily_data['Data']=='01.12.2009']

In [ ]:
Months=np.zeros((len(daily_data['Close']),5))
labels=['Close','Open','High','Low']
for i in tqdm(range(len(daily_data['Close'])-1)):
    k=0
    for j in labels:
        value=float(daily_data[j][len(daily_data[j])-1-i][0])
        value2=float(daily_data[j][len(daily_data[j])-1-i][2:])
        Months[i,k]=value+value2/1000
        k+=1
    l=0
    while daily_data['Variation'][len(daily_data['Variation'])-1-i][l]!=',':
        l+=1
    if daily_data['Variation'][len(daily_data['Variation'])-1-i][0]=='-':
        number1=float(daily_data['Variation'][len(daily_data['Variation'])-1-i][1:l])
    else:
        number1=float(daily_data['Variation'][len(daily_data['Variation'])-1-i][0:l])
    number2=float(daily_data['Variation'][len(daily_data['Variation'])-1-i][l+1:-1])
    Months[i,k]=number1+number2/100
    

In [ ]:
chosen=30#number of days in the history
horizon=5
X_train=np.zeros((len(daily_data['Close'])-chosen-365,chosen,4))
Y_train=np.zeros((len(daily_data['Close'])-chosen-365,2))
X_val=np.zeros((365,chosen,4))
Y_val=np.zeros((365,2))
z=0
print ("Creation of the training data:\n")
for i in tqdm(range(len(daily_data['Close'])-chosen-365-horizon)):
    for j in range(chosen):
        X_train[z,j,0]=Months[i+j,0]
        X_train[z,j,1]=Months[i+j,1]
        X_train[z,j,2]=Months[i+j,2]
        X_train[z,j,3]=Months[i+j,3]
        #X_train[z,j,4]=Months[i+j,4]
    candidates = []
    for k in range(horizon):
        candidates.append(Months[i+chosen+k,2])
    Y_train[z,0]=np.max(candidates)
    candidates = []
    for k in range(horizon):
        candidates.append(Months[i+chosen+k,3])
    Y_train[z,1]=np.min(candidates)
    z+=1
z=0
print ("Creation of the validation data:\n")
for i in tqdm(range(365)):
    for j in range(chosen):
        X_val[z,j,0]=Months[i+j+len(daily_data['Close'])-chosen-365-horizon,0]
        X_val[z,j,1]=Months[i+j+len(daily_data['Close'])-chosen-365-horizon,1]
        X_val[z,j,2]=Months[i+j+len(daily_data['Close'])-chosen-365-horizon,2]
        X_val[z,j,3]=Months[i+j+len(daily_data['Close'])-chosen-365-horizon,3]
        #X_val[z,j,4]=Months[i+j+len(daily_data['Close'])-chosen-365-horizon,4]
    candidates = []
    for k in range(horizon):
        candidates.append(Months[i+len(daily_data['Close'])-365-horizon+k,2])
    Y_val[z,0]=np.max(candidates)
    candidates = []
    for k in range(horizon):
        candidates.append(Months[i+len(daily_data['Close'])-365-horizon+k,3])
    Y_val[z,1]=np.min(candidates)
    z+=1

# LSTM

In [3]:
from keras_tqdm import TQDMNotebookCallback
#del model
chosen=30
#X_train=X_train.reshape((X_train.shape[0],X_train.shape[1],X_train.shape[2]))
#X_val=X_val.reshape((X_val.shape[0],X_val.shape[1],X_val.shape[2]))
#print (X_train.shape)
#print (X_val.shape)
#print (Y_train.shape)
#print (Y_val.shape)
model = Sequential()
#model.add(Conv2D(10,kernel_size=(1,5),strides=(1,5),activation='relu'))
model.add(LSTM(30, return_sequences=1,activation='relu'))
model.add(LSTM(30,activation='relu'))
model.add(Dense(2))
model.compile(optimizer='adam', loss='mse')
#history=model.fit(X_train,Y_train,epochs=20,verbose=0,callbacks=[TQDMNotebookCallback()],shuffle=True,validation_data=(X_val,Y_val))


Instructions for updating:
Colocations handled automatically by placer.


In [ ]:
plt.figure(figsize=(12,4))
epochs=np.arange(20)
plt.plot(epochs,history.history['loss'],label='Train loss')
plt.plot(epochs,history.history['val_loss'],label='Val loss')
plt.grid()
plt.title('Losses')
plt.ylabel('MSE')
plt.xlabel('Epochs')
plt.legend()
plt.show()

In [ ]:
plt.style.use('dark_background')
plt.figure(figsize=(12,6))
days=np.arange(365)
plt.plot(days,Y_val[:,0],label='True behavior high',color='green')
plt.plot(days,Y_val[:,1],label='True behavior low',color='red')
plt.plot(days,model.predict(X_val)[:,0],label='Predicted high',color='lightgreen')
plt.plot(days,model.predict(X_val)[:,1],label='Predicted low',color='orange')
plt.grid()
plt.title('Rendimento reale e predetto del decennale italiano negli ultimi 365 giorni')
plt.ylabel('Rendimento')
plt.xlabel('Giorni ultimo anno')
plt.legend()
plt.savefig('Predizione_2_giorni.pdf')
plt.show()

In [4]:
model.load_weights('model.h5')
daily_data = pd.read_csv("Last.csv",header=None,sep=";")
daily_data.columns=['Data','Close','Open','High','Low']
daily_data.head()

ValueError: You are trying to load a weight file containing 3 layers into a model with 0 layers.

In [ ]:
LM = np.zeros((30,4))
for i in range(30):
    LM[i,0]=float(daily_data['Close'][29-i][0]+'.'+daily_data['Close'][29-i][2:])
    LM[i,1]=float(daily_data['Open'][29-i][0]+'.'+daily_data['Open'][29-i][2:])
    LM[i,2]=float(daily_data['High'][29-i][0]+'.'+daily_data['High'][29-i][2:])
    LM[i,3]=float(daily_data['Low'][29-i][0]+'.'+daily_data['Low'][29-i][2:])

In [ ]:
#model.load_weights('Prediction.h5')
print (model.predict(LM.reshape(1,30,4)))
plt.figure(figsize=(8,4))
plt.plot(LM[25:])
plt.scatter([5],model.predict(LM.reshape(1,30,4))[0,0],color='green')
plt.scatter([5],model.predict(LM.reshape(1,30,4))[0,1],color='red')
plt.grid()
plt.show()